In [1]:
# Import Dependencies
import numpy as np
import pandas as pd

# Import SQL Alchemy
from sqlalchemy import create_engine

# Get the username and password from config file
from config import username, password

import requests
import os
import geopandas as gpd
import seaborn as sns
from shapely.geometry import Point, Polygon
from psycopg2.extensions import register_adapter, AsIs

# Import API key
from config import yelp_api_key

In [2]:
# run extract file
%run Extract.ipynb

# Transformation

## 1) Toronto Restaurant Data

In [3]:
#view restaurant df
restaurant_df['Restaurant Name'].count()

15821

In [4]:
# Remove unwanted columns in restaurant df
cleaned_restaurant_df = restaurant_df[["Category", "Restaurant Name", "Restaurant Price Range", "Restaurant Latitude", "Restaurant Longitude"]].dropna()
cleaned_restaurant_df.reset_index(inplace=True)
cleaned_restaurant_df.drop('index', axis='columns', inplace=True)
cleaned_restaurant_df.head()

,Category,Restaurant Name,Restaurant Price Range,Restaurant Latitude,Restaurant Longitude
0,Afghan,The Host,$11-30,43.669935,-79.395858
1,Afghan,Aanch Modernist Indian Cuisine,$11-30,43.644708,-79.390670
2,Afghan,Silk Road Kabob House,Under $10,43.659816,-79.385591
3,Afghan,Naan & Kabob,$11-30,43.669058,-79.386100
4,Afghan,Afghan Cuisine,$11-30,43.708070,-79.341508


In [5]:
# Read Neighbourhoods data using geopandas
filepath = os.path.join("Resources", "data.csv")
data = pd.read_csv(filepath)
nb = os.path.join('Resources', 'Neighbourhoods', "Neighbourhoods.shp")
regions = gpd.read_file(nb)
regions['neighbourhood'] = regions['FIELD_7'].str.replace(' \(.+\)', '').str.lower()
regions.to_csv('Resources/regions.csv')
regions.head()

,FIELD_1,FIELD_2,FIELD_3,FIELD_4,FIELD_5,FIELD_6,FIELD_7,FIELD_8,FIELD_9,FIELD_10,FIELD_11,FIELD_12,FIELD_13,FIELD_14,FIELD_15,geometry,neighbourhood
0,2101,25886861,25926662,49885,94,94,Wychwood (94),Wychwood (94),None,None,-79.425515,43.676919,16491505,3.217960e+06,7515.779658,"POLYGON ((-79.43592 43.68015, -79.43492 43.680...",wychwood
1,2102,25886820,25926663,49885,100,100,Yonge-Eglinton (100),Yonge-Eglinton (100),None,None,-79.403590,43.704689,16491521,3.160334e+06,7872.021074,"POLYGON ((-79.41096 43.70408, -79.40962 43.704...",yonge-eglinton
2,2103,25886834,25926664,49885,97,97,Yonge-St.Clair (97),Yonge-St.Clair (97),None,None,-79.397871,43.687859,16491537,2.222464e+06,8130.411276,"POLYGON ((-79.39119 43.68108, -79.39141 43.680...",yonge-st.clair
3,2104,25886593,25926665,49885,27,27,York University Heights (27),York University Heights (27),None,None,-79.488883,43.765736,16491553,2.541821e+07,25632.335242,"POLYGON ((-79.50529 43.75987, -79.50488 43.759...",york university heights
4,2105,25886688,25926666,49885,31,31,Yorkdale-Glen Park (31),Yorkdale-Glen Park (31),None,None,-79.457108,43.714672,16491569,1.156669e+07,13953.408098,"POLYGON ((-79.43969 43.70561, -79.44011 43.705...",yorkdale-glen park


In [6]:
# Add new columns, 'neighbourhood_id' and 'neighbourhood_name'
cleaned_restaurant_df['neighbourhood_id'] = 'NaN'
cleaned_restaurant_df['neighbourhood_name'] = 'NaN'

# Assign each restaurant to corresponded neighbourhood polygon using GeoPandas & neighbourhood geometry
for i in range(len(cleaned_restaurant_df)):
    
    lng = cleaned_restaurant_df.loc[i, 'Restaurant Longitude']
    lat = cleaned_restaurant_df.loc[i, 'Restaurant Latitude']
    point = Point(lng, lat)
    
    for j in np.arange(len(regions)):
        poly = regions.loc[j, 'geometry']
        
        if point.within(poly):
            cleaned_restaurant_df.loc[i, 'neighbourhood_id'] = regions.loc[j, 'FIELD_6']
            cleaned_restaurant_df.loc[i, 'neighbourhood_name'] = regions.loc[j, 'neighbourhood']

# Remove the restaurants which were not assigned to any Toronto neighbourhood polygon
cleaned_restaurant_df = cleaned_restaurant_df[cleaned_restaurant_df['neighbourhood_id'] != 'NaN']
cleaned_restaurant_df.head()

,Category,Restaurant Name,Restaurant Price Range,Restaurant Latitude,Restaurant Longitude,neighbourhood_id,neighbourhood_name
0,Afghan,The Host,$11-30,43.669935,-79.395858,95,annex
1,Afghan,Aanch Modernist Indian Cuisine,$11-30,43.644708,-79.390670,77,waterfront communities-the island
2,Afghan,Silk Road Kabob House,Under $10,43.659816,-79.385591,76,bay street corridor
3,Afghan,Naan & Kabob,$11-30,43.669058,-79.386100,75,church-yonge corridor
4,Afghan,Afghan Cuisine,$11-30,43.708070,-79.341508,55,thorncliffe park


### <Final Transformed Table 1: restaurant>
#### cleaned_restaurant_df has duplicated values

In [7]:
# This df has duplicated values 
# (Some restaurants has multiple different categories or one restaurant has multiple locations)
cleaned_restaurant_df

,Category,Restaurant Name,Restaurant Price Range,Restaurant Latitude,Restaurant Longitude,neighbourhood_id,neighbourhood_name
0,Afghan,The Host,$11-30,43.669935,-79.395858,95,annex
1,Afghan,Aanch Modernist Indian Cuisine,$11-30,43.644708,-79.390670,77,waterfront communities-the island
2,Afghan,Silk Road Kabob House,Under $10,43.659816,-79.385591,76,bay street corridor
3,Afghan,Naan & Kabob,$11-30,43.669058,-79.386100,75,church-yonge corridor
4,Afghan,Afghan Cuisine,$11-30,43.708070,-79.341508,55,thorncliffe park
...,...,...,...,...,...,...,...
11532,Waffles,Chinese Egg Waffle Vendor,Under $10,43.652586,-79.398445,78,kensington-chinatown
11534,Waffles,Page One,$11-30,43.657853,-79.376254,75,church-yonge corridor
11535,Waffles,Bread & Butter,Under $10,43.701638,-79.387440,99,mount pleasant east
11536,Waffles,Sugar Miracles,Under $10,43.716805,-79.400696,103,lawrence park south


#### restaurant_df: droped the same restaurants with multiple different categories.

In [8]:
# Dropped the same restaurants with multiple different categories.
restaurant_df = cleaned_restaurant_df.drop_duplicates(subset=['Restaurant Name', 'neighbourhood_id'], keep='first')
restaurant_df.reset_index(inplace=True)
restaurant_df = restaurant_df.drop('index', axis='columns')
restaurant_df

restaurant_transformed = restaurant_df.rename(columns={'Category':'category','Restaurant Name':'restaurant_name',\
                                             'Restaurant Price Range' :'price_range','Restaurant Latitude':'latitude',
                                             'Restaurant Longitude':'longitude'})
index_list = []
for i in range(1, len(restaurant_transformed)+1):
    index_list.append(i)
restaurant_transformed['restaurant_id'] = index_list
restaurant_transformed = restaurant_transformed.set_index('restaurant_id')

In [9]:
restaurant_transformed

,category,restaurant_name,price_range,latitude,longitude,neighbourhood_id,neighbourhood_name
restaurant_id,,,,,,,
1,Afghan,The Host,$11-30,43.669935,-79.395858,95,annex
2,Afghan,Aanch Modernist Indian Cuisine,$11-30,43.644708,-79.390670,77,waterfront communities-the island
3,Afghan,Silk Road Kabob House,Under $10,43.659816,-79.385591,76,bay street corridor
4,Afghan,Naan & Kabob,$11-30,43.669058,-79.386100,75,church-yonge corridor
5,Afghan,Afghan Cuisine,$11-30,43.708070,-79.341508,55,thorncliffe park
...,...,...,...,...,...,...,...
5136,Waffles,Starving Artist,$11-30,43.701535,-79.387209,99,mount pleasant east
5137,Waffles,Wafel Bar,Under $10,43.650571,-79.384568,76,bay street corridor
5138,Waffles,Dazzling Cafe,$11-30,43.761612,-79.409221,51,willowdale east


In [10]:
# Save the restaurant transformed data into csv file
restaurant_transformed.to_csv('clean_data/restaurant.csv')

### <Final Transformed Table 2: neighbourhood_restaurant>

In [11]:
# number of restaurants for each neighbourhood
neighbourhood_restaurant = restaurant_df.groupby(['neighbourhood_id','neighbourhood_name'])['Restaurant Name'].count()
neighbourhood_restaurant.sort_values(ascending=False, inplace=True)
neighbourhood_restaurant = neighbourhood_restaurant.reset_index()
neighbourhood_restaurant.set_index('neighbourhood_id', inplace=True)
neighbourhood_restaurant.rename(columns={'Restaurant Name':'number_of_restaurants'}, inplace=True)
neighbourhood_restaurant

,neighbourhood_name,number_of_restaurants
neighbourhood_id,,
76,bay street corridor,355
77,waterfront communities-the island,354
78,kensington-chinatown,280
75,church-yonge corridor,214
95,annex,180
...,...,...
112,beechborough-greenbrook,2
8,humber heights-westmount,2
13,etobicoke west mall,2


In [12]:
# Save the neighbourhood restaurant transformed data into csv file
neighbourhood_restaurant.to_csv('clean_data/neighbourhood_restaurant.csv')

## 2) Toronto Neighbourhood Data

In [13]:
# neighbourhood_name
neighbourhood['neighbourhood_name'] = neighbourhood['AREA_NAME'].str.replace(' \(.+\)', '')

# Rename column name
neighbourhood = neighbourhood.rename(columns={'AREA_SHORT_CODE':'neighbourhood_id'})

# Remove unwanted columns
neighbourhood = neighbourhood[["neighbourhood_id", "neighbourhood_name"]]

# Set index as neighbourhood_id
neighbourhood = neighbourhood.set_index('neighbourhood_id')
neighbourhood = neighbourhood.sort_index()

### <Final Transfomred Table 3: neighbourhood>

In [14]:
neighbourhood

,neighbourhood_name
neighbourhood_id,
1,West Humber-Clairville
2,Mount Olive-Silverstone-Jamestown
3,Thistletown-Beaumond Heights
4,Rexdale-Kipling
5,Elms-Old Rexdale
...,...
136,West Hill
137,Woburn
138,Eglinton East


## 3) Toronto Ethnicity Data

In [15]:
# Put filters on Characteristic to get all Ethnicities in Toronto neighbourhoods
characteristic_df = ethnicity_df.loc[((ethnicity_df["Characteristic"] == " North American Aboriginal origins") | (ethnicity_df["Characteristic"] == " Other North American origins") | (ethnicity_df["Characteristic"] == " European origins") | (ethnicity_df["Characteristic"] == " Caribbean origins") | (ethnicity_df["Characteristic"] == " Latin; Central and South American origins") | (ethnicity_df["Characteristic"] == " African origins") | (ethnicity_df["Characteristic"] == " Asian origins") | (ethnicity_df["Characteristic"] == " Oceania origins"))]

# Remove unwanted columns
for col in characteristic_df.columns:
    if col == '_id' or col == 'Category' or col == 'Topic' or col == 'Data Source':
        del characteristic_df[col]
        

# Did a transpose of all cities columns to 1 single column named as neighbourhood name
transposed_df = characteristic_df.loc[:,'Agincourt North':'Yorkdale-Glen Park']
ethnicity_transformed = transposed_df.transpose().reset_index()

ethnicity_transformed.columns =["neighbourhood_name","oceania_origins","asian_origins",\
                    "north_american_aboriginal_origins","other_north_american_origins",\
                    "latin_origins","european_origins","african_origins","caribbean_origins"]

# To add neighbourhood_id column
neighbourhood_info = pd.read_csv('Resources/Neighbourhoods.csv')

# neighbourhood_name
neighbourhood_info['neighbourhood_name'] = neighbourhood_info['AREA_NAME'].str.replace(' \(.+\)', '')

# Rename column name
neighbourhood_info = neighbourhood_info.rename(columns={'AREA_SHORT_CODE':'neighbourhood_id'})

# Remove unwanted columns
neighbourhood_info = neighbourhood_info[["neighbourhood_id", "neighbourhood_name"]]

# Merge data with neighbourhood data to expose neighbourhood_id in Ethnicity table
ethnicity = pd.merge(ethnicity_transformed, neighbourhood_info, on = 'neighbourhood_name', how = "inner")

# Set Index as Neighbourhood id
ethnicity_transformed = ethnicity.set_index('neighbourhood_id')

In [16]:
#Remove commas in each ethnicity column as it is an integer value
ethnicity_transformed['asian_origins'] = ethnicity_transformed['asian_origins'].str.replace("," , '')
ethnicity_transformed['north_american_aboriginal_origins'] = ethnicity_transformed['north_american_aboriginal_origins'].str.replace("," , '')
ethnicity_transformed['other_north_american_origins'] = ethnicity_transformed['other_north_american_origins'].str.replace("," , '')
ethnicity_transformed['latin_origins'] = ethnicity_transformed['latin_origins'].str.replace("," , '')
ethnicity_transformed['european_origins'] = ethnicity_transformed['european_origins'].str.replace("," , '')
ethnicity_transformed['african_origins'] = ethnicity_transformed['african_origins'].str.replace("," , '')
ethnicity_transformed['caribbean_origins'] = ethnicity_transformed['caribbean_origins'].str.replace("," , '')

### <Final Transfomred Table 4: ethnicity>

In [17]:
ethnicity_transformed

,neighbourhood_name,oceania_origins,asian_origins,north_american_aboriginal_origins,other_north_american_origins,latin_origins,european_origins,african_origins,caribbean_origins
neighbourhood_id,,,,,,,,,
129,Agincourt North,10,24305,40,1345,470,3055,535,1445
128,Agincourt South-Malvern West,0,17955,105,1190,480,3770,625,1395
20,Alderwood,0,2055,305,2355,315,9135,215,350
95,Annex,140,6485,475,5255,765,21055,1040,750
42,Banbury-Don Mills,20,12025,230,3230,585,13435,990,815
...,...,...,...,...,...,...,...,...,...
94,Wychwood,90,2500,335,2010,645,9685,610,740
100,Yonge-Eglinton,50,2895,140,2695,370,8455,310,280
97,Yonge-St.Clair,80,2330,215,2525,300,9460,370,295


In [18]:
# Save the ethnicity transformed data into csv file
ethnicity_transformed.to_csv('clean_data/ethnicity.csv')

## 4) Toronto Neighbourhood Income Data

In [19]:
# Put filters on Category and Topic to get Average and Median Income per household
income_df = income_df.loc[(income_df["Category"] == 'Income') & (income_df["Topic"]  == 'Income of households')
& ((income_df["Attribute"] == 'Median household total income $')|(income_df["Attribute"]  == 'Average household total income $'))]

# Did transpose of all neighbourhood columns to 1 single column
transposed_df = income_df.loc[:,'Agincourt North':'Yorkdale-Glen Park']
income_data = transposed_df.transpose().reset_index()
income_data.columns = ["neighbourhood_name","median_income","average_income"]

# Using neighbourhood informantion to add neighbourhood_id column by merging
neighbourhood = pd.read_csv('Resources/Neighbourhoods.csv')

# To merge the DataFrames, add the neighbourhood_column
neighbourhood['neighbourhood_name'] = neighbourhood['AREA_NAME'].str.replace(' \(.+\)', '')
income_data['neighbourhood_name'] = income_data['neighbourhood_name'].str.replace(' \(.+\)', '')

# Merging income_data and neighbourhood 
income_neighbourhood = pd.merge(income_data, neighbourhood,  on='neighbourhood_name')

# Remove unwated columns
income = income_neighbourhood[['AREA_SHORT_CODE', 'neighbourhood_name', 'median_income', 'average_income']]

# Rename columns
income = income.rename(columns={'AREA_SHORT_CODE':'neighbourhood_id'})

# Set index with 'neighbourhood_id' column
income = income.set_index('neighbourhood_id')
income_transformed = income.sort_index()

### <Final Transfomred Table 5: income>

In [20]:
income_transformed

,neighbourhood_name,median_income,average_income
neighbourhood_id,,,
1,West Humber-Clairville,66241.0,76228.0
2,Mount Olive-Silverstone-Jamestown,49934.0,58605.0
3,Thistletown-Beaumond Heights,62042.0,73512.0
4,Rexdale-Kipling,56545.0,66781.0
5,Elms-Old Rexdale,50846.0,63201.0
...,...,...,...
136,West Hill,49713.0,63461.0
137,Woburn,52018.0,63651.0
138,Eglinton East,46495.0,58035.0


## 5) Toronto Neighbourhood Crime Data

In [21]:
# Remove unwanted columns
crime_df = crime_df.loc[:,["Neighbourhood", "Hood_ID", "Assault_Rate_2019", "AutoTheft_Rate_2019", "BreakandEnter_Rate_2019", "Homicide_Rate_2019", "Robbery_Rate_2019", "TheftOver_Rate_2019"]]

# Calculate total averate crime rates and create a new column for the values
# Crime Rates: Rate of crimes for 2019 per 100,000 population 
crime_df["total_average_crime_rate"] = round((crime_df["Assault_Rate_2019"] + crime_df["AutoTheft_Rate_2019"] + crime_df["BreakandEnter_Rate_2019"] +
                                       crime_df["Homicide_Rate_2019"] + crime_df["Robbery_Rate_2019"] + crime_df["TheftOver_Rate_2019"]) / 6, 2)

# Remove unwanted columns
crime = crime_df.loc[:,["Neighbourhood", "Hood_ID", "total_average_crime_rate"]]


# Rename the columns names
crime = crime.rename(columns={"Hood_ID" : "neighbourhood_id",
                              "Neighbourhood" : "neighbourhood_name"
                             })

# Set index as neighbourhood_id
crime = crime.set_index('neighbourhood_id')
crime_transformed = crime.sort_index()

### <Final Transfomred Table 6: income>

In [22]:
crime_transformed

,neighbourhood_name,total_average_crime_rate
neighbourhood_id,,
1,West Humber-Clairville,507.32
2,Mount Olive-Silverstone-Jamestown,232.13
3,Thistletown-Beaumond Heights,236.50
4,Rexdale-Kipling,245.35
5,Elms-Old Rexdale,216.80
...,...,...
136,West Hill,383.95
137,Woburn,206.32
138,Eglinton East,231.97


## 6) Restaurants Rating & Review Data from Yelp API

In [23]:
# Drop the duplicated rows
yelp_rating = rating_df.drop_duplicates(subset=['id', 'name', 'ratings', 'review_counts', 'zip_code'], keep='first')

# Rename columns
yelp_rating = yelp_rating.rename(columns={'id':'restaurant_id', 'name':'restaurant_name'})
yelp_rating_transformed = yelp_rating.set_index('restaurant_id')

### <Final Transformed Table 7: neighbourhood_restaurant>

In [24]:
yelp_rating_transformed

,restaurant_name,category,ratings,review_counts,zip_code
restaurant_id,,,,,
e41TP5cXZqSrz50xCBJqZw,Insomnia Restaurant & Lounge,Lounges,4.0,923,M5S 1Y6
r_BrIgzYcwo1NAuG9dLbpg,Pai Northern Thai Kitchen,Thai,4.5,2895,M5H 3G8
Uq-GOs9_IqweUsB5MdII9w,Emma's Country Kitchen,Breakfast & Brunch,4.0,394,M6C 1B6
iGEvDk6hsizigmXhDKs2Vg,Seven Lives Tacos y Mariscos,Mexican,4.5,1323,M5T 2K1
-ICGmF2qUVKdvOehVNgPbg,Lamesa Filipino Kitchen,Filipino,4.0,352,M6C 1A9
...,...,...,...,...,...
RNdcUG1sCTLdUo8dEC9NJw,The Local,Bars,3.5,58,M6R 2M9
kbSSGo6zRPSdBT-CwG2cNg,Suvaiyakam Restaurant,Sri Lankan,4.5,11,M1W 3G5
jyPsc5xUFpVOuuSylPkZdw,Dumplings & Szechuan Cuisine,Chinese,3.5,25,M1W 2H7


In [25]:
# Save the yelp rating transformed data into csv file
yelp_rating_transformed.to_csv('clean_data/yelp_ratings.csv')